In [1]:
import numpy as np
from typing import List, Tuple, Dict, Set

In [3]:
def insert_in_center(target_list, list_to_insert):
    insert_position = len(target_list) // 2
    target_list[insert_position:insert_position] = list_to_insert
    return target_list

In [4]:
def make_step(
        B: List[int], 
        Omega: Set[int], 
        D: List[List[Set[int]]], 
        beta: Set[int],
) -> Tuple[List[int], Set[int], List[List[Set[int]]], Set[int], bool]:
    # Строим новое множество Omega для данного шага
    Omega = Omega & D[list(beta)[0] - 1][list(beta)[1] - 1] - beta
    
    # Eсли новое множество Omega пустое, то возвращаем предыдущее состояние и флаг False
    if not Omega:
        return B, Omega, D, beta, False
    
    else:
        # Строим новое множество D для данного шага
        D: List[List[Set[int]]] = [[set() for _ in range(len(A))] for _ in range(len(A))]
        for i in Omega:
            for j in Omega:
                if i != j:
                    current_D = E[i-1] & H[j-1] & Omega
                    if {i, j} & current_D:
                        D[i-1][j-1] = current_D
                        
        # Строим новое множество S для данного шага
        S = [
            {x[0], x[1]} 
            for x in sorted(
                [
                    (i + 1, j + 1, D[i][j]) 
                    for i in range(len(D)) 
                    for j in range(len(D[i])) 
                    if D[i][j]
                ], 
                key=lambda x: len(x[2])
            )
        ]
        
        # Eсли новое множество S пустое, то возвращаем флаг False
        if not S:
            return insert_in_center(B, list(Omega)[:1]), Omega, D, beta, False
        
        # Иначе возвращаем новое состояние
        else:
            return insert_in_center(B, list(S[0])), Omega, D, beta, True

In [5]:
#-----=================================================-----
# Исходная матрица A
# A: np.ndarray = np.array([
#     [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
#     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
#     [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
#     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
#     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
#     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
#     [1, 1, 1, 1, 1, 1, 0, 0, 1, 1],
#     [1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
#     [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
#     [1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
# ])

A: np.ndarray = np.array([
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 0, 1],
    [1, 1, 1, 1, 1, 0],
])

In [6]:
# Множества B1 и B2
B1 = []
B2 = []

# Множества E и H
E: List[Set[int]] = [set(np.where(row == 0)[0] + 1) for row in A]
H: List[Set[int]] = [set(np.where(col == 0)[0] + 1) for col in A.transpose()]
print("Множества E:", E)
print("Множества H:", H)

# Множество Omega^{1,0}
Omega_1_0 = set(np.where(A.diagonal() == 0)[0] + 1)
print("Множество Omega^{1,0}:", Omega_1_0)

# Строим множества D^{(1,0)}_{(i,j)}
D_1_0: List[List[Set[int]]] = [[set() for _ in range(len(A))] for _ in range(len(A))]
for i in Omega_1_0:
    for j in Omega_1_0:
        if i != j:
            current_D = E[i-1] & H[j-1] & Omega_1_0
            if {i, j} & current_D:
                D_1_0[i-1][j-1] = current_D
print("Множества D^{(1,0)}:", D_1_0)

# Строим множество S^{(1,0)} (все индексы непустых множеств D^{1,0}_{i,j}, отсортированные по мощности)
S_1_0 = [
    {x[0], x[1]} 
    for x in sorted(
        [
            (i + 1, j + 1, D_1_0[i][j]) 
            for i in range(len(D_1_0)) 
            for j in range(len(D_1_0[i])) 
            if D_1_0[i][j]
        ], 
        key=lambda x: len(x[2])
    )
]
print("Множество S^{(1,0)}:", S_1_0)

for beta in S_1_0:
    print("Узловой элемент beta^{(1, 0)}:", beta)
    
    # Вставляем beta^(1,0) в B1
    B1 = insert_in_center(B1, list(beta))
    print("Начальное значение B1 для данного beta:", B1)
    
    # Копируем множество Omega^{1,0} в Omega и множество D^{(1,0)} в D
    Omega = Omega_1_0
    D = D_1_0
    
    #Начинаем цикл построения B1
    flag = True
    while flag:
        B1, Omega, D, beta, flag = make_step(B1, Omega, D, beta)
    print("Итоговое значение B1 для данного beta:", B1)
    
    # Удаляем ненужные переменные
    del flag, Omega, D, beta
    
    # Строим Omega^{2,0}
    Omega_2_0 = Omega_1_0 - set(B1)
    print("Начальное значение Omega^{2,0} для данного beta:", Omega_2_0)
    
    # Строим множества D^{(2,0)}_{(i,j)}
    D_2_0: List[List[Set[int]]] = [[set() for _ in range(len(A))] for _ in range(len(A))]
    for i in Omega_2_0:
        for j in Omega_2_0:
            if i != j:
                current_D = E[i-1] & H[j-1] & Omega_2_0
                if {i, j} & current_D:
                    D_2_0[i-1][j-1] = current_D
                    
    print("Множества D^{(2,0)}:", D_2_0)
    
    # Строим множество S^{(2,0)} (все индексы непустых множеств D^{2,0}_{i,j}, отсортированные по мощности)
    S_2_0 = [
        {x[0], x[1]} 
        for x in sorted(
            [
                (i + 1, j + 1, D_2_0[i][j]) 
                for i in range(len(D_2_0)) 
                for j in range(len(D_2_0[i])) 
                if D_2_0[i][j]
            ], 
            key=lambda x: len(x[2])
        )
    ]
    
    if S_2_0:
        for beta in S_2_0:
            print("Узловой элемент beta^{(2, 0)}:", beta)

            # Вставляем beta^(2,0) в B2
            B2 = insert_in_center(B2, list(beta))
            print("Начальное значение B2 для данного beta:", B2)
            
            # Копируем множество Omega^{2,0} в Omega и множество D^{(2,0)} в D
            Omega = Omega_2_0
            D = D_2_0
            
            #Начинаем цикл построения B2
            flag = True
            while flag:
                B2, Omega, D, beta, flag = make_step_for_B1(B2, Omega, D, beta)
            print("Итоговое значение B2 для данного beta:", B2)
            
            # Удаляем ненужные переменные
            del flag, Omega, D, beta
        
    else:
        B2 = insert_in_center(B2, list(Omega_2_0)[:1])
        print("Итоговое значение B2 для данного beta:", B2)

Множества E: [{1, 2, 3, 5, 6}, {1, 2, 3, 4, 6}, {1, 2, 3, 4, 5}, set(), {5}, {6}]
Множества H: [{1, 2, 3}, {1, 2, 3}, {1, 2, 3}, {2, 3}, {1, 3, 5}, {1, 2, 6}]
Множество Omega^{1,0}: {1, 2, 3, 5, 6}
Множества D^{(1,0)}: [[set(), {1, 2, 3}, {1, 2, 3}, set(), {1, 3, 5}, {1, 2, 6}], [{1, 2, 3}, set(), {1, 2, 3}, set(), set(), {1, 2, 6}], [{1, 2, 3}, {1, 2, 3}, set(), set(), {1, 3, 5}, set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()]]
Множество S^{(1,0)}: [{1, 2}, {1, 3}, {1, 5}, {1, 6}, {1, 2}, {2, 3}, {2, 6}, {1, 3}, {2, 3}, {3, 5}]
Узловой элемент beta^{(1, 0)}: {1, 2}
Начальное значение B1 для данного beta: [1, 2]
Итоговое значение B1 для данного beta: [1, 3, 2]
Начальное значение Omega^{2,0} для данного beta: {5, 6}
Множества D^{(2,0)}: [[set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), set(), set()], [set(), set(), set(), set(), s